### Introduction
This Notebook shows how to scrape the goals and penalty data from the [EIHL](www.eliteleague.co.uk) website. The data are put into Pandas DataFrames. This data is used to perform analysis on the shot data also available from the EIHL website, which is performed in another Notebook. The accompanying blog post can be found [here](www.ncalvert.uk/posts/eihlshots/).

First, let's import the libraries that we'll need.

In [1]:
import datetime
import matplotlib.pyplot as plt
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
import numpy as np
import pandas as pd
import progressbar
import re
import requests
import time
%matplotlib inline


### Download the Gamesheets
The gamesheets are hosted online at https://eihlhq.co.uk/pdf/print/de-html/MATCH_ID where the MATCH_ID is a unique numeric ID for each game. I'm listing the IDs here, as I scraped them previously in my other Notebook.

In [85]:
match_ids = [
    1120, 1065,953,1032,1161,1024,912,1136,1073,1008,969,1049,990,928,2327,1086,1069,1028,949,1004,
    1141,965,1053,1116,1045,973,1100,924,1012,1157,925,1013,1156,1044,972,1101,964,1117,1005,933,
    1140,1091,2310,1029,1087,1068,1048,968,987,1009,1137,944,1072,1025,1033,1121,1064,952,959,1038,
    1096,1079,934,1002,1147,1055,963,1110,975,1043,1106,1014,922,1151,1126,955,1063,1171,1034,902,
    1167,914,1022,1130,1075,2337,1059,980,979,996,1018,1019,2320,997,1058,981,2336,1131,1074,1166,
    1023,1170,1035,903,1127,954,1062,1015,923,1150,1042,1107,1054,962,1111,935,1003,1146,1097,1078,
    919,1039,1081,958,1153,1016,998,1104,1041,977,1112,961,1057,1145,936,1094,1169,1082,1128,1108,
    994,1149,1077,1098,1132,1020,916,1165,900,1036,1173,1061,957,1124,1060,1125,901,1037,1021,917,
    1164,1076,1099,1133,1148,2334,983,1109,995,1083,1129,1168,1095,1144,1001,937,1113,960,1056,999,
    1040,976,1152,1017,1159,992,1118,984,2333,1071,947,1134,910,1026,1163,1030,951,1067,1122,1088,
    1155,1010,1102,971,1047,988,1114,1051,1143,930,1006,1138,1092,1084,1085,1139,1093,931,1007,989,
    1115,1050,1103,1046,950,1066,1123,1089,1031,907,911,1027,1162,1070,946,1135,1119,985,1158
]

Iterate through the ```match_ids``` and download the gamesheet. There's a 10 second delay between each download to ensure that the website does not become overloaded/they don't think I'm performing a DDOS attack on them.

In [39]:
m_ids = []
match_htmls = []
for m_id in match_ids:
    try:
        url = "https://eihlhq.co.uk/pdf/print/de-html/" + str(m_id)
        match_htmls.append(requests.get(url).text)
        m_ids.append(m_id)
    except:
        print("Failed: " + str(m_id))
    time.sleep(10)

Got ID: 1120
Got ID: 1065
Got ID: 953
Got ID: 1032
Got ID: 1161
Got ID: 1024
Got ID: 912
Got ID: 1136
Got ID: 1073
Got ID: 1008
Got ID: 969
Got ID: 1049
Got ID: 990
Got ID: 928
Got ID: 2327
Got ID: 1086
Got ID: 1069
Got ID: 1028
Got ID: 949
Got ID: 1004
Got ID: 1141
Got ID: 965
Got ID: 1053
Got ID: 1116
Got ID: 1045
Got ID: 973
Got ID: 1100
Got ID: 924
Got ID: 1012
Got ID: 1157
Got ID: 925
Got ID: 1013
Got ID: 1156
Got ID: 1044
Got ID: 972
Got ID: 1101
Got ID: 964
Got ID: 1117
Got ID: 1005
Got ID: 933
Got ID: 1140
Got ID: 1091
Got ID: 2310
Got ID: 1029
Got ID: 1087
Got ID: 1068
Got ID: 1048
Got ID: 968
Got ID: 987
Got ID: 1009
Got ID: 1137
Got ID: 944
Got ID: 1072
Got ID: 1025
Got ID: 1033
Got ID: 1121
Got ID: 1064
Got ID: 952
Got ID: 959
Got ID: 1038
Got ID: 1096
Got ID: 1079
Got ID: 934
Got ID: 1002
Got ID: 1147
Got ID: 1055
Got ID: 963
Got ID: 1110
Got ID: 975
Got ID: 1043
Got ID: 1106
Got ID: 1014
Got ID: 922
Got ID: 1151
Got ID: 1126
Got ID: 955
Got ID: 1063
Got ID: 1171
Got ID: 1

### Get the Penalty and Goal Data

The gamesheets are interpreted by Pandas using the ```read_html``` function. This results in an array of DataFrames relating to different parts of the gamesheet. We're interested in the home goals DataFrame (```df[5]```), the home penalties DataFrame (```df[6]```), and the corresponding away goals & penalties (```df[11]``` & ```df[12]```).

The goal data was the same for all gamesheets, and so was straightforward to combine. The penalty data varied depending on whether there was a bench penalty or a netminder penalty. In these cases there was an extra column denoting which player served the penalty and this required some extra manipulation. I also had to correct for the penalties that were penalty shots rather than minor/major penalties.

The penalty and goal data for each team and each game were put in separate DataFrames before further processing, rather than combining into a single large penalty & a single large goal DataFrame which is done later on.

In [331]:
homegoaldfs = []
awaygoaldfs = []
homependfs = []
awaypendfs = []
for m in progressbar.progressbar(match_htmls):
    df = pd.read_html(m)
    hg = df[5].dropna(axis=0,how="all").reset_index()
    hg.columns = hg.columns.droplevel(level=0)
    homegoaldfs.append(hg)
    hp = df[6].dropna(axis=0,how="all").reset_index()
    if type(hp.columns)==pd.core.indexes.base.Index:
        hp.rename(
            columns = {
                'Penalties':"Time",
                'Penalties.1':"No.",
                'Penalties.2':"Min",
                'Penalties.3':"Penalty",
                'Penalties.4':"Start",
                'Penalties.5':"End",
                'Penalties.6':"Served"
            },
            inplace=True
        )
        hp.drop(0,inplace=True)
    else:
         hp.columns = hp.columns.droplevel(level=0)
    if(len(hp)>0):
        hp.loc[:,"isPS"] = False
        hp.loc[hp.loc[:,"Min"]=="PS","isPS"] = True
        hp.loc[hp.loc[:,"Min"]=="PS","Min"] = "-1"
        hp.loc[:,"Min"] = hp.loc[:,"Min"].map(int)
    homependfs.append(hp)
    ag = df[11].dropna(axis=0,how="all").reset_index()
    ag.columns = ag.columns.droplevel(level=0)
    awaygoaldfs.append(ag)
    ap = df[12].dropna(axis=0,how="all").reset_index()
    if type(ap.columns)==pd.core.indexes.base.Index:
        ap.rename(
            columns = {
                'Penalties':"Time",
                'Penalties.1':"No.",
                'Penalties.2':"Min.",
                'Penalties.3':"Penalty",
                'Penalties.4':"Start",
                'Penalties.5':"End",
                'Penalties.6':"Served"
            },
            inplace=True
        )
        ap.drop(0,inplace=True)
    else:
         ap.columns = ap.columns.droplevel(level=0)
    if(len(ap)>0):
        ap.loc[:,"isPS"] = False
        ap.loc[ap.loc[:,"Min."]=="PS","isPS"] = True
        ap.loc[ap.loc[:,"Min."]=="PS","Min."] = "-1"
        ap.loc[:,"Min."] = ap.loc[:,"Min."].map(int)
    awaypendfs.append(ap)

N/A% (0 of 239) |                        | Elapsed Time: 0:00:00 ETA:  --:--:--/Users/ncalvertuk/opt/anaconda3/lib/python3.7/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)
100% (239 of 239) |######################| Elapsed Time: 0:02:30 Time:  0:02:30


### Correcting The Penalty Data

Some of the penalty data was incorrect, this was because there were several penalties given at once. In a real game situation each team must have a minimum of 3 skaters on the ice at each time. If 3 penalties occur at the same time, then the final penalty will start after the first 2 have expired. I have corrected these values manually as there were only a few situations where this occurred.

In [468]:
# Fix some of the sheets:

homependfs[59].loc[13,"Start"] = "50:17"
homependfs[59].loc[13,"End"] = "52:17"

awaypendfs[82].loc[8,"Start"] = "54:05"
awaypendfs[82].loc[8,"End"] = "54:05"

awaypendfs[190].loc[5,"Start"] = "20:32"
awaypendfs[190].loc[5,"End"] = "22:32"

awaypendfs[192].loc[2,"End"] = "12:23"
awaypendfs[192].loc[3,"Start"] = "12:05"
awaypendfs[192].loc[3,"End"] = "14:05"

awaypendfs[192].loc[2,"End"] = "12:23"

awaypendfs[230].loc[3,"End"] = "39:16"
awaypendfs[230].loc[4,"End"] = "39:46"
awaypendfs[230].loc[5,"Start"] = "39:16"
awaypendfs[230].loc[5,"End"] = "41:16"
awaypendfs[230].loc[6,"Start"] = "39:46"
awaypendfs[230].loc[6,"End"] = "41:29"

### Process the Penalty Data

We'll now further process the data, to get the start and end times of the penalties in seconds

In [469]:
def getpentime(x):
    try:
        return int(x[0:2])*60 + int(x[3:])
    except:
        return np.nan

In [470]:
allpendfs = [];
for idx in range(0,len(awaypendfs)):
    ap = awaypendfs[idx].copy()
    hp = homependfs[idx].copy()
    ap.rename(columns={"Min.":"Min"},inplace=True)
    if len(ap)>0:
        ap.loc[:,"Home"] = False
        ap.loc[:,"Start_sec"] = ap.loc[:,"Start"].apply(getpentime)
        ap.loc[:,"End_sec"] = ap.loc[:,"End"].apply(getpentime)
        ap.loc[:,"Offset"] = False
    if len(hp)>0:
        hp.loc[:,"Home"] = True
        hp.loc[:,"Start_sec"] = hp.loc[:,"Start"].apply(getpentime)
        hp.loc[:,"End_sec"] = hp.loc[:,"End"].apply(getpentime)
        hp.loc[:,"Offset"] = False
    hp = hp.append(ap, ignore_index=True)
    allpendfs.append(hp)
    allpendfs[idx].loc[:,"Match_ID"] = match_ids[idx]
    

### Process the Goal Data

Now we'll process the goal data. In particular, we'll convert the time into seconds (the shot data times are in seconds), and we'll also get the number of players on the ice for the home and away teams when each goal was scored. New columns are also created which will later contain information about whether a goal is a delayed penalty goal and whether it is a penalty shot.

In [498]:
def goalnplyrs(gs,home,homegame=True):
    try:
        if home == homegame:
            return int(gs[0])
        else:
            return int(gs[2])
    except:
        return np.nan

In [502]:
allgoaldfs = [];
for idx in range(0,len(awaygoaldfs)):
    ag = awaygoaldfs[idx].copy()
    hg = homegoaldfs[idx].copy()
    if len(ag)>0:
        ag.loc[:,"Home"] = False
        ag.loc[:,"Goal_sec"] = ag.loc[:,"Time"].apply(getpentime)
        ag.loc[:,"isDPG"] = False
        ag.loc[:,"isPS"] = False
    if len(hg)>0:
        hg.loc[:,"Home"] = True
        hg.loc[:,"Goal_sec"] = hg.loc[:,"Time"].apply(getpentime)
        hg.loc[:,"isDPG"] = False
        hg.loc[:,"isPS"] = False
    hg = hg.append(ag, ignore_index=True)
    allgoaldfs.append(hg)
    allgoaldfs[idx].loc[:,"Match_ID"] = match_ids[idx]
    allgoaldfs[idx]["HomePlayers"] = allgoaldfs[idx].apply(
        lambda x: goalnplyrs(gs = x['GS'], home = x['Home'], homegame=True),
        axis=1
    )
    allgoaldfs[idx]["AwayPlayers"] = allgoaldfs[idx].apply(
        lambda x: goalnplyrs(gs = x['GS'], home = x['Home'], homegame=False),
        axis=1
    )


Here's what one of the Goal DataFrames looks like.

In [506]:
allgoaldfs[200]

,,#,Time,G,A1,A2,GS,P1,P2,P3,...,N4,N5,N6,Home,Goal_sec,isDPG,isPS,Match_ID,HomePlayers,AwayPlayers
0,0,1.0,02:16,18.0,20.0,5.0,5/5,23.0,18.0,20.0,...,9.0,2.0,NaN,True,136,False,False,1155,5,5
1,1,2.0,19:27,8.0,5.0,45.0,4/4,18.0,45.0,5.0,...,21.0,NaN,NaN,True,1167,False,False,1155,4,4
2,2,3.0,37:46,27.0,51.0,5.0,5/4,5.0,8.0,27.0,...,7.0,NaN,NaN,True,2266,False,False,1155,5,4
3,3,4.0,59:36,90.0,27.0,51.0,5/5,20.0,44.0,90.0,...,17.0,20.0,95.0,True,3576,False,False,1155,5,5
4,0,1.0,47:09,74.0,24.0,95.0,5/5,27.0,20.0,43.0,...,19.0,29.0,NaN,False,2829,False,False,1155,5,5


### Calculating offsetting Penalties

Some penalties are offsetting which means that teams play 5-on-5 despite each having a player (or more) in the penalty box. Later on we'll calculate the number of skaters on the ice at all times, so we have to calculate which penalties are offsetting to ensure that we get the correct numbers.

In [472]:
for i in range(0,len(allpendfs)):
    for tpen in allpendfs[i].loc[:,"Start_sec"].unique():
        n_home = len(
            allpendfs[i].loc[
                (allpendfs[i].loc[:,"Start_sec"] == tpen) &
                (allpendfs[i].loc[:,"Home"] == True),
                "Start_sec"
            ]
        )
        n_away = len(
            allpendfs[i].loc[
                (allpendfs[i].loc[:,"Start_sec"] == tpen) &
                (allpendfs[i].loc[:,"Home"] == False),
                "Start_sec"
            ]
        )
        if (n_home >0) & (n_away >0):
            home_idxs = np.where(
                (allpendfs[i].loc[:,"Start_sec"] == tpen) &
                (allpendfs[i].loc[:,"Home"] == True)
            )[0]
            away_idxs = np.where(
                (allpendfs[i].loc[:,"Start_sec"] == tpen) &
                (allpendfs[i].loc[:,"Home"] == False)
            )[0]
            for hi in home_idxs:
                for ai in away_idxs:
                    if ((allpendfs[i].loc[hi,"End_sec"] == allpendfs[i].loc[ai,"End_sec"])&
                    (allpendfs[i].loc[hi,"Offset"] == False)&
                    (allpendfs[i].loc[ai,"Offset"] == False)):
                        allpendfs[i].loc[hi,"Offset"] = True
                        allpendfs[i].loc[ai,"Offset"] = True

Let's also get the data on which goals are penalty shot goals and which are delayed penalty goals. Penalty shots have a penalty length of -1 in the penalty data, and delayed penalty goals have a penalty length of 0 in the penalty data and should match up in time with the goal.

In [473]:
for i in range(0,len(awaygoaldfs)):
    # PS
    if sum(allpendfs[i].loc[:,"Min"]<0) >0:
        idx = np.where(allpendfs[i].loc[:,"Min"]<0)[0]
        for ind in idx:
            t = allpendfs[i].loc[allpendfs[i].index[ind],"Time"]
            allgoaldfs[i].loc[allgoaldfs[i].loc[:,"Time"]==t,"isPS"] = True
    # DPG
    if sum(allpendfs[i].loc[:,"Min"]==0) >0:
        idx = np.where(allpendfs[i].loc[:,"Min"]==0)[0]
        for ind in idx:
            t = allpendfs[i].loc[allpendfs[i].index[ind],"Time"]
            allgoaldfs[i].loc[allgoaldfs[i].loc[:,"Time"]==t,"isDPG"] = True

### Combine the DataFrames

We'll now combine the DataFrames, so we have a single goal DataFrame and a single Penalty DataFrame.

In [474]:
allgoaldf = pd.DataFrame()
allpendf = pd.DataFrame()
for i in range(0,len(allgoaldfs)):
    allgoaldf = allgoaldf.append(allgoaldfs[i])
    allpendf = allpendf.append(allpendfs[i])
allpendf=allpendf.reset_index()
allgoaldf=allgoaldf.reset_index()

This next block of code calculates the number of players on the ice at each time for every game (split into home and away team). This will be used in the other notebook to calculate how many skaters are in the ice when each shot is taken.

In [476]:
t = np.arange(0,3600,1)
hps = np.zeros([len(t),allpendf.loc[:,"Match_ID"].nunique()])
aps = np.zeros([len(t),allpendf.loc[:,"Match_ID"].nunique()])
for idx in range(0,allpendf.loc[:,"Match_ID"].nunique()):
    m_id = allpendf.loc[:,"Match_ID"].unique()[idx]
    if (sum((allpendf.loc[:,"Match_ID"]==m_id)&(allpendf.loc[:,"Home"]==True))>0):
        hpdf = allpendf.loc[(allpendf.loc[:,"Match_ID"]==m_id)&(allpendf.loc[:,"Home"]==True),:].copy()
        h_ids = np.where(((hpdf.loc[:,"Min"] == 2.)|(hpdf.loc[:,"Min"] == 5.))&(hpdf.loc[:,"Offset"]==False))[0]
        for h_id in h_ids:
            start_sec = int(hpdf.loc[hpdf.index[h_id],"Start_sec"])
            end_sec = int(hpdf.loc[hpdf.index[h_id],"End_sec"])
            hps[start_sec:end_sec,idx] +=1
    if (sum((allpendf.loc[:,"Match_ID"]==m_id)&(allpendf.loc[:,"Home"]==False))>0):
        apdf = allpendf.loc[(allpendf.loc[:,"Match_ID"]==m_id)&(allpendf.loc[:,"Home"]==False),:].copy()
        a_ids = np.where(((apdf.loc[:,"Min"] == 2.)|(apdf.loc[:,"Min"] == 5.))&(apdf.loc[:,"Offset"]==False))[0]
        for a_id in a_ids:
            start_sec = int(apdf.loc[apdf.index[a_id],"Start_sec"])
            end_sec = int(apdf.loc[apdf.index[a_id],"End_sec"])
            aps[start_sec:end_sec,idx] +=1
    
   
            
hnplayers = 5.*np.ones_like(hps) - hps

anplayers = 5.*np.ones_like(aps) - aps

Save the penalty DataFrame to a ```.pkl``` file so that we can load it when processing the shot data.

In [512]:
allpendf.to_pickle("./AllPenalties.pkl")

### Empty Net Goals

Empty net goals are quite common, often when a team is behind by a goal (or sometimes 2) with a couple of minutes remaining in the game then they will replace their netminder with a skater. Often this results in an empty net goal, as there is no netminder to make the save. This information was not clear from the gamesheets, so I had to scrape it from the Event Data for each game. This is done below.

In [70]:
# Now we need to see which goals are empty net goals
fixtures_url = "https://www.eliteleague.co.uk/schedule?id_season=2&id_team=0&id_month=999"
fixtures_html = requests.get(fixtures_url)
idxs = [m.start() for m in re.finditer("a href=\"/game/", fixtures_html.text)]
game_ids = []
base_ids = []
# ind = idxs[0]
# ind1 = fixtures_html.text.find(" class",ind)
# g_id = fixtures_html.text[ind+8:ind1-1]
all_eng_times = []
for ind in idxs:
    ind1 = fixtures_html.text.find(" class",ind)
    g_id = fixtures_html.text[ind+8:ind1-1]
    track_url = "https://www.eliteleague.co.uk" +g_id + "/tracking"
    b_id = re.findall("\d+",g_id)[0]
    track_html = requests.get(track_url)
    ind2 = track_html.text.find("https://eihl.hokejovyzapis.cz/visualization/")
    ind3 = track_html.text.find("\"",ind2)
    s_id = track_html.text[(ind3-4):ind3]
    if(s_id[0] == "="):
        s_id = s_id[1:]
    if s_id not in game_ids:
        game_ids.append(int(s_id))
        base_ids.append(int(b_id))
    game_url = "https://www.eliteleague.co.uk" + g_id
    game_html = requests.get(game_url)
    eng_times = []
    engidxs = [m.start() for m in re.finditer("Empty net Goal", game_html.text)]
    for engid in engidxs:
        eng_time = game_html.text[engid-185:(engid-180)]
        eng_times.append(eng_time)
    all_eng_times.append(eng_times)
    time.sleep(1)

In [82]:
allgoaldf.loc[:,"isENG"] = False
for i in range(0,len(game_ids)):
    if len(all_eng_times[i])>0:
        same_match = allgoaldf.loc[:,"Match_ID"] == game_ids[i]
        for j in range(0,len(all_eng_times[i])):
            same_time = allgoaldf.loc[:,"Time"] == all_eng_times[i][j]
            allgoaldf.loc[(same_match&same_time),"isENG"] = True

In [83]:
allgoaldf.head()

,index,,#,Time,G,A1,A2,GS,P1,P2,...,N3,N4,N5,N6,Home,Goal_sec,isDPG,isPS,Match_ID,isENG
0,0,0,1.0,26:39,37.0,9.0,75.0,5/5,37.0,25.0,...,28.0,6.0,15.0,NaN,True,1599.0,False,False,1120.0,False
1,1,1,2.0,31:54,16.0,10.0,NaN,5/4,16.0,11.0,...,15.0,6.0,NaN,NaN,True,1914.0,False,False,1120.0,False
2,2,2,3.0,32:26,75.0,11.0,9.0,5/5,75.0,37.0,...,6.0,37.0,28.0,NaN,True,1946.0,False,False,1120.0,False
3,0,0,1.0,01:50,48.0,20.0,21.0,5/5,28.0,21.0,...,59.0,58.0,22.0,NaN,True,110.0,False,False,1065.0,False
4,1,1,2.0,51:13,41.0,28.0,NaN,5/5,41.0,27.0,...,19.0,21.0,59.0,NaN,True,3073.0,False,False,1065.0,False


Save the goals DataFrame to a ```.pkl``` file for later processing.

In [84]:
allgoaldf.to_pickle("./AllGoals.pkl")